In [1]:
from datasets import load_dataset

dataset = load_dataset("oscar", language="hu", split="train")

Using custom data configuration unshuffled_deduplicated_hu-language=hu
Reusing dataset oscar (C:\Users\ayber\.cache\huggingface\datasets\oscar\unshuffled_deduplicated_hu-language=hu\0.0.0\84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2)


In [2]:
chars_to_ignore_regex = """[",", "?", "¿", ".", "!", "¡", ";", "；", ":", '""', "%", '"', "�", "ʿ", "·", "჻", "~", "՞",
                   "؟", "،", "।", "॥", "«", "»", "„", "“", "”", "「", "」", "‘", "’", "《", "》", "(", ")", "[", "]",
                   "{", "}", "=", "`", "_", "+", "<", ">", "…", "–", "°", "´", "ʾ", "‹", "›", "©", "®", "—", "→", "。",
                   "、", "﹂", "﹁", "‧", "～", "﹏", "，", "｛", "｝", "（", "）", "［", "］", "【", "】", "‥", "〽",
                   "『", "』", "〝", "〟", "⟨", "⟩", "〜", "：", "！", "？", "♪", "؛", "/", "\\", "º", "−", "^", "ʻ", "ˆ"]"""

In [3]:
import re

def extract_text(batch):
  text = batch["text"]
  batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
  return batch

In [4]:
dataset = dataset.map(extract_text, remove_columns="id")

Loading cached processed dataset at C:\Users\ayber\.cache\huggingface\datasets\oscar\unshuffled_deduplicated_hu-language=hu\0.0.0\84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2\cache-dfe897d68b82c1cf.arrow


In [5]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\ayber/.huggingface/token


In [6]:
dataset.push_to_hub(f"hu_oscar_processed", split="train")

The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/37 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [8]:
len(dataset["text"])

6582908

In [9]:
with open("text50k.txt", "w") as file:
  file.write(" ".join(dataset["text"][:50000]))

UnicodeEncodeError: 'charmap' codec can't encode character '\u0151' in position 14: character maps to <undefined>

In [ ]:
from collections import defaultdict


def frequency_dicts(file):
    """Creates frequency dictionaries and returns them. Dictionaries set up as
    {uni/(bi/tri/quadgram) : probability(_probs) or count(_freqs)}."""

    quadgram_freqs = defaultdict(int)  # tracks hard count
    #quadgram_probs = defaultdict(float)  # tracks probabilities
    trigram_freqs = defaultdict(int)  # tracks hard count
    #trigram_probs = defaultdict(float)  # tracks probabilities
    bigram_freqs = defaultdict(int)  # tracks hard count
    #bigram_probs = defaultdict(float)  # tracks probabilities
    word_freqs = defaultdict(int)  # tracks hard count
    #word_probs = defaultdict(float)  # tracks probabilities
    tokenizer = nltk.RegexpTokenizer(r'[A-Za-z]+[\S]?[A-Za-z0-9]+|[A-Za-z]+|\d+[a-z]+|\d+.?\d+|\d+')

    with open(file) as f:
        for line in f:  # allows for tokenization of each individual line
            text = line.rstrip().lower()
            tokens = tokenizer.tokenize(text)

            for index in range(0, len(tokens) - 3):  # creates quadgram frequency dictionary
                quadgram_freqs[(tokens[index], tokens[index + 1], tokens[index + 2], tokens[index + 3])] += 1
            for index in range(0, len(tokens) - 2):  # creates trigram frequency dictionary
                trigram_freqs[(tokens[index], tokens[index + 1], tokens[index + 2])] += 1
            for index in range(0, len(tokens) - 1):  # creates bigram frequency dictionary
                bigram_freqs[(tokens[index], tokens[index + 1])] += 1
            for token in tokens:  # creates unigram frequency dictionary
                word_freqs[token] += 1

    #tot_words = sum(word_freq.values())

    #for k in word_freq.keys():  # creates unigram probabilities
        #word_probs[k] = word_freq[k] / tot_words

    #tot_bis = sum(bigram_freqs.values())

    #for k in bigram_freqs.keys():  # creates bigram probabilities
        #bigram_probs[k] = bigram_freqs[k] / tot_bis

    #tot_tris = sum(trigram_freqs.values())

    #for k in trigram_freqs.keys():  # creates trigram probabilities
        #trigram_probs[k] = trigram_freqs[k] / tot_tris

    #tot_quads = sum(quadgram_freqs.values())

    #for k in quadgram_freqs.keys():  # creates quadgram probabilities
        #quadgram_probs[k] = quadgram_freqs[k] / tot_quads

    return word_freqs, bigram_freqs, trigram_freqs, quadgram_freqs

In [ ]:
uni, bi, tri, quuad = frequency_dicts("C:/Users/ayber/PycharmProjects/ASR-for-less-resourced-languages/text.txt")